# Data Preparation for MSc Project

In [ ]:
%pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

### Import Libraries

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

### Setup Mediapipe models and functions

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def mediapipe_Detection(image, model):
      #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #set flag to False
        image.flags.writeable = False
        #Detection 
        results = model.process(image)
        #RGB to BGR
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #set flag to True
        image.flags.writeable = True

        return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
         mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
         mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=2 ))

In [5]:
def extract_keypoints(results):
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([lh,rh])
    return rh

### Create Folders and Data Path

In [6]:
#path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# actions list
actions = np.array(['forward', 'backward', 'left', 'right', 'spin', 'stop', 'rest']) 
# actions = np.array(['hello']) 

#30 videos of data
no_sequence = 30

# 30 frames each
sequence_length = 30

In [ ]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

### Capture Data Points

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #loop through actions
    for action in actions:
        #loop through sequences aka videos
        for sequence in range(no_sequence):
            #loop through frames
            for frame_num in range(sequence_length):
                #Read Feed
                ret, frame = cap.read()

                image, results = mediapipe_Detection(frame, holistic)
                draw_landmarks(image,results)

                #apply wait logic
                if frame_num == 0:
                    cv2.putText(image,'Collecting Frames for {} video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,
                    (0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(300)
                    cv2.putText(image,'STARTING_COLLECTION',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA)
                else:
                    cv2.putText(image,'Collecting Frames for {} video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,
                    (0,0,255),1,cv2.LINE_AA)

                #Export Keypoints
                keypoints = extract_keypoints(results)
                np_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(np_path,keypoints)
                 
                #show to screen
                cv2.imshow('Webcam Feed', image)

                #end loopq
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

cap.release()
cv2.destroyAllWindows()

In [39]:
cap.release()
cv2.destroyAllWindows()

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequence):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
x= np.array(sequences) 
x.shape

(210, 30, 63)

In [12]:
y = to_categorical(labels).astype(int)
y.shape

(210, 7)

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05)

In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True,activation='relu'))
model.add(LSTM(64, return_sequences=False,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [21]:
res = [.7,0.2,0.1]

In [22]:
actions[np.argmax(res)]

'forward'

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [24]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
7/7 [==============================] - 3s 20ms/step - loss: 1.9362 - categorical_accuracy: 0.1457
Epoch 2/2000
7/7 [==============================] - 0s 19ms/step - loss: 1.8919 - categorical_accuracy: 0.1910
Epoch 3/2000
7/7 [==============================] - 0s 18ms/step - loss: 1.7319 - categorical_accuracy: 0.3819
Epoch 4/2000
7/7 [==============================] - 0s 19ms/step - loss: 1.6351 - categorical_accuracy: 0.3920
Epoch 5/2000
7/7 [==============================] - 0s 19ms/step - loss: 1.5109 - categorical_accuracy: 0.3417
Epoch 6/2000
7/7 [==============================] - 0s 19ms/step - loss: 1.5561 - categorical_accuracy: 0.4020
Epoch 7/2000
7/7 [==============================] - 0s 19ms/step - loss: 1.4729 - categorical_accuracy: 0.4523
Epoch 8/2000
7/7 [==============================] - 0s 19ms/step - loss: 1.4135 - categorical_accuracy: 0.4774
Epoch 9/2000
7/7 [==============================] - 0s 19ms/step - loss: 1.4102 - categorical_accuracy: 0.4824
E

7/7 [==============================] - 0s 19ms/step - loss: 0.1815 - categorical_accuracy: 0.9698
Epoch 75/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.1476 - categorical_accuracy: 0.9648
Epoch 76/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.1896 - categorical_accuracy: 0.9296
Epoch 77/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.1007 - categorical_accuracy: 0.9698
Epoch 78/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.1026 - categorical_accuracy: 0.9698
Epoch 79/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.1170 - categorical_accuracy: 0.9648
Epoch 80/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.1616 - categorical_accuracy: 0.9598
Epoch 81/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.2371 - categorical_accuracy: 0.9196
Epoch 82/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.3195 - categorical_accuracy: 0.9045
Epoch 

7/7 [==============================] - 0s 20ms/step - loss: 0.0391 - categorical_accuracy: 0.9899
Epoch 147/2000
7/7 [==============================] - 0s 19ms/step - loss: 0.0421 - categorical_accuracy: 0.9899
Epoch 148/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0783 - categorical_accuracy: 0.9749
Epoch 149/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.1699 - categorical_accuracy: 0.9598
Epoch 150/2000
7/7 [==============================] - 0s 22ms/step - loss: 0.3698 - categorical_accuracy: 0.9196
Epoch 151/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.3521 - categorical_accuracy: 0.8744
Epoch 152/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.1845 - categorical_accuracy: 0.9648
Epoch 153/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.2722 - categorical_accuracy: 0.9648
Epoch 154/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.2211 - categorical_accuracy: 0.934

7/7 [==============================] - 0s 19ms/step - loss: 0.0372 - categorical_accuracy: 0.9899
Epoch 219/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.0337 - categorical_accuracy: 0.9899
Epoch 220/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0365 - categorical_accuracy: 0.9899
Epoch 221/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.0297 - categorical_accuracy: 0.9899
Epoch 222/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0303 - categorical_accuracy: 0.9899
Epoch 223/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0282 - categorical_accuracy: 0.9899
Epoch 224/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.0291 - categorical_accuracy: 0.9899
Epoch 225/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.0534 - categorical_accuracy: 0.9849
Epoch 226/2000
7/7 [==============================] - 0s 21ms/step - loss: 0.0381 - categorical_accuracy: 0.984

7/7 [==============================] - 0s 21ms/step - loss: 0.0192 - categorical_accuracy: 0.9950
Epoch 291/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0192 - categorical_accuracy: 0.9950
Epoch 292/2000
7/7 [==============================] - 0s 19ms/step - loss: 0.0187 - categorical_accuracy: 0.9950
Epoch 293/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0182 - categorical_accuracy: 0.9950
Epoch 294/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0173 - categorical_accuracy: 0.9950
Epoch 295/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0178 - categorical_accuracy: 0.9950
Epoch 296/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0177 - categorical_accuracy: 0.9950
Epoch 297/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0176 - categorical_accuracy: 0.9950
Epoch 298/2000
7/7 [==============================] - 0s 20ms/step - loss: 0.0173 - categorical_accuracy: 0.995

...

7/7 [==============================] - 0s 19ms/step - loss: 3.2274e-05 - categorical_accuracy: 1.0000
Epoch 1719/2000
7/7 [==============================] - 0s 21ms/step - loss: 3.2087e-05 - categorical_accuracy: 1.0000
Epoch 1720/2000
7/7 [==============================] - 0s 21ms/step - loss: 3.1908e-05 - categorical_accuracy: 1.0000
Epoch 1721/2000
7/7 [==============================] - 0s 20ms/step - loss: 3.1648e-05 - categorical_accuracy: 1.0000
Epoch 1722/2000
7/7 [==============================] - 0s 21ms/step - loss: 3.1415e-05 - categorical_accuracy: 1.0000
Epoch 1723/2000
7/7 [==============================] - 0s 20ms/step - loss: 3.1245e-05 - categorical_accuracy: 1.0000
Epoch 1724/2000
7/7 [==============================] - 0s 20ms/step - loss: 3.0968e-05 - categorical_accuracy: 1.0000
Epoch 1725/2000
7/7 [==============================] - 0s 20ms/step - loss: 3.0781e-05 - categorical_accuracy: 1.0000
Epoch 1726/2000
7/7 [==============================] - 0s 20ms/step - lo

7/7 [==============================] - 0s 20ms/step - loss: 2.0785e-05 - categorical_accuracy: 1.0000
Epoch 1788/2000
7/7 [==============================] - 0s 21ms/step - loss: 2.0668e-05 - categorical_accuracy: 1.0000
Epoch 1789/2000
7/7 [==============================] - 0s 21ms/step - loss: 2.0545e-05 - categorical_accuracy: 1.0000
Epoch 1790/2000
7/7 [==============================] - 0s 21ms/step - loss: 2.0408e-05 - categorical_accuracy: 1.0000
Epoch 1791/2000
7/7 [==============================] - 0s 19ms/step - loss: 2.0286e-05 - categorical_accuracy: 1.0000
Epoch 1792/2000
7/7 [==============================] - 0s 22ms/step - loss: 2.0138e-05 - categorical_accuracy: 1.0000
Epoch 1793/2000
7/7 [==============================] - 0s 20ms/step - loss: 2.0015e-05 - categorical_accuracy: 1.0000
Epoch 1794/2000
7/7 [==============================] - 0s 21ms/step - loss: 1.9900e-05 - categorical_accuracy: 1.0000
Epoch 1795/2000
7/7 [==============================] - 0s 21ms/step - lo

7/7 [==============================] - 0s 20ms/step - loss: 1.3856e-05 - categorical_accuracy: 1.0000
Epoch 1857/2000
7/7 [==============================] - 0s 20ms/step - loss: 1.3788e-05 - categorical_accuracy: 1.0000
Epoch 1858/2000
7/7 [==============================] - 0s 20ms/step - loss: 1.3703e-05 - categorical_accuracy: 1.0000
Epoch 1859/2000
7/7 [==============================] - 0s 21ms/step - loss: 1.3646e-05 - categorical_accuracy: 1.0000
Epoch 1860/2000
7/7 [==============================] - 0s 20ms/step - loss: 1.3562e-05 - categorical_accuracy: 1.0000
Epoch 1861/2000
7/7 [==============================] - 0s 20ms/step - loss: 1.3477e-05 - categorical_accuracy: 1.0000
Epoch 1862/2000
7/7 [==============================] - 0s 21ms/step - loss: 1.3444e-05 - categorical_accuracy: 1.0000
Epoch 1863/2000
7/7 [==============================] - 0s 20ms/step - loss: 1.3343e-05 - categorical_accuracy: 1.0000
Epoch 1864/2000
7/7 [==============================] - 0s 22ms/step - lo

7/7 [==============================] - 0s 21ms/step - loss: 9.5561e-06 - categorical_accuracy: 1.0000
Epoch 1926/2000
7/7 [==============================] - 0s 21ms/step - loss: 9.5172e-06 - categorical_accuracy: 1.0000
Epoch 1927/2000
7/7 [==============================] - 0s 20ms/step - loss: 9.4609e-06 - categorical_accuracy: 1.0000
Epoch 1928/2000
7/7 [==============================] - 0s 20ms/step - loss: 9.4208e-06 - categorical_accuracy: 1.0000
Epoch 1929/2000
7/7 [==============================] - 0s 21ms/step - loss: 9.3711e-06 - categorical_accuracy: 1.0000
Epoch 1930/2000
7/7 [==============================] - 0s 21ms/step - loss: 9.3255e-06 - categorical_accuracy: 1.0000
Epoch 1931/2000
7/7 [==============================] - 0s 22ms/step - loss: 9.2932e-06 - categorical_accuracy: 1.0000
Epoch 1932/2000
7/7 [==============================] - 0s 20ms/step - loss: 9.2399e-06 - categorical_accuracy: 1.0000
Epoch 1933/2000
7/7 [==============================] - 0s 19ms/step - lo

7/7 [==============================] - 0s 21ms/step - loss: 6.6989e-06 - categorical_accuracy: 1.0000
Epoch 1995/2000
7/7 [==============================] - 0s 21ms/step - loss: 6.6648e-06 - categorical_accuracy: 1.0000
Epoch 1996/2000
7/7 [==============================] - 0s 21ms/step - loss: 6.6366e-06 - categorical_accuracy: 1.0000
Epoch 1997/2000
7/7 [==============================] - 0s 21ms/step - loss: 6.6049e-06 - categorical_accuracy: 1.0000
Epoch 1998/2000
7/7 [==============================] - 0s 21ms/step - loss: 6.5749e-06 - categorical_accuracy: 1.0000
Epoch 1999/2000
7/7 [==============================] - 0s 21ms/step - loss: 6.5360e-06 - categorical_accuracy: 1.0000
Epoch 2000/2000
7/7 [==============================] - 0s 19ms/step - loss: 6.5006e-06 - categorical_accuracy: 1.0000


In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            32768     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 7)                 231       
                                                                 
Total params: 187,463
Trainable params: 187,463
Non-tr

In [26]:
res = model.predict(x_test)

1/1 [==============================] - 0s 238ms/step


In [44]:
# actions[np.argmax(res[0])]
actions[np.argmax(res[0])]

'forward'

In [34]:
actions[np.argmax(y_train[0])]

'left'

In [28]:
model.save('action.h5')

In [29]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [55]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 14ms/step


In [57]:
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

AxisError: axis 1 is out of bounds for array of dimension 1

In [32]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[ 9,  0],
        [ 0,  2]],

       [[ 8,  1],
        [ 0,  2]],

       [[10,  0],
        [ 0,  1]],

       [[ 9,  0],
        [ 0,  2]],

       [[ 8,  0],
        [ 1,  2]],

       [[10,  0],
        [ 0,  1]]])

In [33]:
accuracy_score(ytrue, yhat)

0.9090909090909091

In [35]:
from scipy import stats

In [48]:
colors = [(245,117,16), (117,245,16), (16,117,245), (255, 255, 102),(51, 204, 255),(0, 51, 102),(255, 153, 0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [58]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_Detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Webcam Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x5558b0554cc0) is not the object's thread (0x5558b009c910).
Cannot move to target thread (0x5558b0554cc0)

QObject::moveToThread: Current thread (0x5558b0554cc0) is not the object's thread (0x5558b009c910).
Cannot move to target thread (0x5558b0554cc0)

QObject::moveToThread: Current thread (0x5558b0554cc0) is not the object's thread (0x5558b009c910).
Cannot move to target thread (0x5558b0554cc0)

QObject::moveToThread: Current thread (0x5558b0554cc0) is not the object's thread (0x5558b009c910).
Cannot move to target thread (0x5558b0554cc0)

QObject::moveToThread: Current thread (0x5558b0554cc0) is not the object's thread (0x5558b009c910).
Cannot move to target thread (0x5558b0554cc0)

QObject::moveToThread: Current thread (0x5558b0554cc0) is not the object's thread (0x5558b009c910).
Cannot move to target thread (0x5558b0554cc0)

QObject::moveToThread: Current thread (0x5558b0554cc0) is not the object's thread (0x5558b009c910).
Cannot move to tar

1/1 [==============================] - 0s 12ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 12ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0

1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 15ms/step
spin
1/1 [==============================] - 0s 15ms/step
spin
1/1 [==============================] - 0s 14ms/step
spin
1/1 [==============================] - 0s 24ms/step
spin
1/1 [==============================] - 0s 13ms/step
left
1/1 [==========================

1/1 [==============================] - 0s 12ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 12ms/step
left
1/1 [==============================] - 0s 11ms/s

1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0s 11ms/step
forward
1/1 [==============================] - 0

1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 14ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 11ms/step
right
1/1 [==============================] - 0s 12ms/step
right
1/1 [==============================] - 0s 12ms/step
right
1/1 [=========

1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 12ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 12ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 12ms/step
left
1/1 [==============================] - 0s 11ms/step
left
1/1 [==============================] - 0s 12ms/step
left
1/1 [==========================

1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 12ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 12ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 11ms/step
stop
1/1 [==============================] - 0s 12ms/step
stop
1/1 [==========================

1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 12ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 12ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 12ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [===================

1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 13ms/step
backward
1/1 [==============================] - 0s 12ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 11ms/step
backward
1/1 [==============================] - 0s 12ms/step
backward
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 12ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 11ms/step
spin
1/1 [==============================] - 0s 11ms/step
left
1/1 [==

In [53]:
cap.release()
cv2.destroyAllWindows()